In [ ]:
import pandas as pd
import numpy as np
from kan import *
import torch
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

import geopandas as gpd
import cartopy.crs as ccrs
import matplotlib.patches as mpatches
import cartopy.feature as cfeature

In [ ]:
plt.rcParams.update({
    'font.size': 12,               # Font size
    'axes.labelsize': 15,          # Label size
    'axes.titlesize': 15,          # Title size
    'axes.linewidth': 1.2,         # Axis line width
    'xtick.labelsize': 10,         # Size of x-axis tick labels
    'ytick.labelsize': 10,         # Size of y-axis tick labels
    'xtick.major.size': 6,         # Length of major ticks on the x-axis
    'ytick.major.size': 6,         # Length of major ticks on the y-axis
    'xtick.major.width': 1.0,      # Width of major ticks on the x-axis
    'ytick.major.width': 1.0,      # Width of major ticks on the y-axis
    'figure.dpi': 300,             # Image resolution
    'savefig.dpi': 300,            # Resolution for saving figures
    'figure.figsize': (12, 5),     # Figure size
})

In [ ]:
df = pd.read_csv('hurricanes_with_sea_temperature.csv')
storms = pd.read_csv('storms.csv')
storms['category'] = storms['category'].fillna(0).astype(int)
storms['date'] = pd.to_datetime(storms[['year', 'month', 'day']])
storms = storms[storms['date'] >= '1981-09-01']
storms = storms[['lat', 'long', 'category', 'year', 'month', 'day', 'status']]

In [ ]:
storms = storms.reset_index(drop=True)
df['status'] = storms['status']
df_filtered = df[df['status'].isin(['tropical storm', 'hurricane', 'tropical depression'])]
df_filtered['status'] = df_filtered['status'].replace({'tropical depression': 0, 'tropical storm': 1, 'hurricane': 2})

In [ ]:
percentil_95 = np.percentile(df_filtered['sst_value'].dropna(), 95)
storms_filtered = df_filtered[(df_filtered['sst_value'] > percentil_95)]

In [ ]:
storms_filtered = storms_filtered[(storms_filtered['long'] >= -100) & 
                                  (storms_filtered['long'] <= -75) &
                                  (storms_filtered['lat'] >= 17.5) & 
                                  (storms_filtered['lat'] <= 35)]


In [ ]:
tropical_depression = storms_filtered[storms_filtered['status'] == 0]
tropical_storm = storms_filtered[storms_filtered['status'] == 1]
hurricane = storms_filtered[storms_filtered['status'] == 2]

In [ ]:
dataset_1 = storms_filtered.copy()
dataset_1['extra'] = (dataset_1['status'] > 0).astype(int)

dataset_2 = storms_filtered.copy()
dataset_2['extra'] = (dataset_2['status'] > 1).astype(int)

# Tropical Depression

In [ ]:
y_train_app = np.array(dataset_1['extra'])
x_train_app = np.array(dataset_1[['long','lat']])

x_train_app[:, 0] = (x_train_app[:, 0] + 180) / 360
x_train_app[:, 1] = (x_train_app[:, 1] + 90) / 180

In [ ]:
x_train_tensor = torch.tensor(x_train_app, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_app, dtype=torch.long)

model = KAN(width=[2, 2], grid=2, k=3)

dataset = {
    'train_input': x_train_tensor,
    'train_label': y_train_tensor,
    'test_input': x_train_tensor,
    'test_label': y_train_tensor
}

model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss());

In [ ]:
x_min, x_max = x_train_app[:, 0].min()-0.001, x_train_app[:, 0].max()
y_min, y_max = x_train_app[:, 1].min(), x_train_app[:, 1].max()
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 1000),
                     np.linspace(y_min, y_max, 1000))

grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

with torch.no_grad():
    Z = model(grid_tensor)
    Z = torch.softmax(Z, dim=1)  
    probabilities1 = Z[:, 1].reshape(xx.shape)

latitudes = 180 * yy - 90
longitudes = 360 * xx - 180

tectonic_plates = gpd.read_file('tectonicplates-master/PB2002_plates.shp')

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_xlim(longitudes.min(), longitudes.max())
ax.set_ylim(latitudes.min(), latitudes.max())

im = ax.pcolormesh(longitudes, latitudes, 1 - probabilities1, shading='auto', cmap='RdBu_r', vmin=0, vmax=1, transform=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND, zorder=2) 
ax.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=2)
ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, zorder=2)
ax.add_feature(cfeature.LAKES, alpha=0.5, zorder=2)
ax.add_feature(cfeature.RIVERS, zorder=2)

gl = ax.gridlines(draw_labels=True, linewidth=0.2, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

tectonic_plates.plot(ax=ax, edgecolor='lightgray', facecolor='none', linewidth=0.5, transform=ccrs.PlateCarree(), zorder=3)

aux_rect = mpatches.Rectangle(
    xy=(0, 0), 
    width=1, height=1, 
    transform=ax.transAxes, 
    linewidth=0.5, edgecolor='black', facecolor='none', zorder=5  
)
ax.add_patch(aux_rect)

ax.scatter(hurricane['long'], hurricane['lat'], color='blue', s=1, alpha=0.35, edgecolor='k', zorder=4)
ax.scatter(tropical_storm['long'], tropical_storm['lat'], color='blue', s=1, alpha=0.35, edgecolor='k', zorder=4)
ax.scatter(tropical_depression['long'], tropical_depression['lat'], color='red', s=4, alpha=1, edgecolor='k', zorder=4)

ax.set_title(f'Tropical Depression', fontsize=16)

cbar = fig.colorbar(im, ax=ax, orientation='horizontal')

plt.show()

In [ ]:
data = np.column_stack((x_train_app[:, 0], x_train_app[:, 1]))
data_tensor = torch.tensor(data, dtype=torch.float32)

with torch.no_grad():
    Z_data = model(data_tensor)
    Z_data = torch.softmax(Z_data, dim=1)
    probabilities_data_1 = Z_data[:, 1]

prob_c1 = 1 - probabilities_data_1

# Tropical Storm

In [ ]:
y_train_app = np.array(dataset_2['extra'])
x_train_app = np.array(dataset_2[['long','lat']])

x_train_app[:, 0] = (x_train_app[:, 0] + 180) / 360
x_train_app[:, 1] = (x_train_app[:, 1] + 90) / 180

In [ ]:
x_train_tensor = torch.tensor(x_train_app, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_app, dtype=torch.long)

model = KAN(width=[2, 2], grid=2, k=3)

dataset = {
    'train_input': x_train_tensor,
    'train_label': y_train_tensor,
    'test_input': x_train_tensor,
    'test_label': y_train_tensor
}

model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss());

In [ ]:
x_train_tensor = torch.tensor(x_train_app, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_app, dtype=torch.long)

model = KAN(width=[2, 2], grid=2, k=3)

dataset = {
    'train_input': x_train_tensor,
    'train_label': y_train_tensor,
    'test_input': x_train_tensor,
    'test_label': y_train_tensor
}

model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss());

In [ ]:
x_min, x_max = x_train_app[:, 0].min()-0.001, x_train_app[:, 0].max()
y_min, y_max = x_train_app[:, 1].min(), x_train_app[:, 1].max()
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 1000),
                     np.linspace(y_min, y_max, 1000))

grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

with torch.no_grad():
    Z = model(grid_tensor)
    Z = torch.softmax(Z, dim=1)  
    probabilities2 = Z[:, 1].reshape(xx.shape)

latitudes = 180 * yy - 90
longitudes = 360 * xx - 180

tectonic_plates = gpd.read_file('tectonicplates-master/PB2002_plates.shp')

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_xlim(longitudes.min(), longitudes.max())
ax.set_ylim(latitudes.min(), latitudes.max())

im = ax.pcolormesh(longitudes, latitudes, probabilities1 - probabilities2, shading='auto', cmap='RdBu_r', vmin=0, vmax=1, transform=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND, zorder=2) 
ax.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=2)
ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, zorder=2)
ax.add_feature(cfeature.LAKES, alpha=0.5, zorder=2)
ax.add_feature(cfeature.RIVERS, zorder=2)

gl = ax.gridlines(draw_labels=True, linewidth=0.2, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

tectonic_plates.plot(ax=ax, edgecolor='lightgray', facecolor='none', linewidth=0.5, transform=ccrs.PlateCarree(), zorder=3)

aux_rect = mpatches.Rectangle(
    xy=(0, 0), 
    width=1, height=1, 
    transform=ax.transAxes, 
    linewidth=0.5, edgecolor='black', facecolor='none', zorder=5  
)
ax.add_patch(aux_rect)

ax.scatter(hurricane['long'], hurricane['lat'], color='blue', s=1, alpha=0.35, edgecolor='k', zorder=4)
ax.scatter(tropical_storm['long'], tropical_storm['lat'], color='red', s=4, alpha=1, edgecolor='k', zorder=4)
ax.scatter(tropical_depression['long'], tropical_depression['lat'], color='blue', s=1, alpha=0.35, edgecolor='k', zorder=4)

ax.set_title(f'Tropical Storm', fontsize=16)

cbar = fig.colorbar(im, ax=ax, orientation='horizontal')

plt.show()

In [ ]:
data = np.column_stack((x_train_app[:, 0], x_train_app[:, 1]))
data_tensor = torch.tensor(data, dtype=torch.float32)

with torch.no_grad():
    Z_data = model(data_tensor)
    Z_data = torch.softmax(Z_data, dim=1)
    probabilities_data_2 = Z_data[:, 1]

prob_c2 =  probabilities_data_1 - probabilities_data_2

# Hurricane

In [ ]:
fig = plt.figure(figsize=(10, 5))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_xlim(longitudes.min(), longitudes.max())
ax.set_ylim(latitudes.min(), latitudes.max())

im = ax.pcolormesh(longitudes, latitudes, probabilities2, shading='auto', cmap='RdBu_r', vmin=0, vmax=1, transform=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND, zorder=2) 
ax.add_feature(cfeature.COASTLINE, linewidth=0.5, zorder=2)
ax.add_feature(cfeature.BORDERS, linestyle=':', linewidth=0.5, zorder=2)
ax.add_feature(cfeature.LAKES, alpha=0.5, zorder=2)
ax.add_feature(cfeature.RIVERS, zorder=2)

gl = ax.gridlines(draw_labels=True, linewidth=0.2, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

tectonic_plates.plot(ax=ax, edgecolor='lightgray', facecolor='none', linewidth=0.5, transform=ccrs.PlateCarree(), zorder=3)

aux_rect = mpatches.Rectangle(
    xy=(0, 0), 
    width=1, height=1, 
    transform=ax.transAxes, 
    linewidth=0.5, edgecolor='black', facecolor='none', zorder=5  
)
ax.add_patch(aux_rect)

ax.scatter(hurricane['long'], hurricane['lat'], color='red', s=4, alpha=1, edgecolor='k', zorder=4)
ax.scatter(tropical_storm['long'], tropical_storm['lat'], color='blue', s=1, alpha=0.35, edgecolor='k', zorder=4)
ax.scatter(tropical_depression['long'], tropical_depression['lat'], color='blue', s=1, alpha=0.35, edgecolor='k', zorder=4)

ax.set_title(f'Hurricane', fontsize=16)

cbar = fig.colorbar(im, ax=ax, orientation='horizontal')

plt.show()

## Data preparation for Dunn-Smyth Residuals

In [ ]:
df = pd.DataFrame({
    'y_true': storms_filtered['status'],
    'class_0': prob_c1.numpy(),
    'class_1': prob_c2.numpy(),
    'class_2': probabilities_data_2.numpy()
})

df.to_csv('residuals_app2.csv', index=False) 